# MLP Clustering Significance Stability over Multiple Trainings - Different #Clusters

### Before Running on Perceptron

Go to `src/train_nn.py` to line 46, and change the path `./models/` to `/scratch/<your username>/models`

Do not forget to undo this change!

DO NOT COMMIT THIS CHANGE TO GITHUB!!!

In [1]:
N_CLUSTERS = 4

In [2]:
# Parameters
N_CLUSTERS = 10


In [3]:
%load_ext autoreload
%autoreload 1

In [4]:
import sys
sys.path.append('..')

import random
import subprocess
from multiprocessing import Pool

import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from tqdm import tqdm, trange
from IPython import display

from src.visualization import draw_mlp_clustering_report, run_double_spectral_cluster, run_spectral_cluster
from src.utils import get_weights_paths, build_clustering_results, cohen_d_stats
from src.experiment_tagging import get_model_path, MODEL_TAG_LOOKUP
# from src.spectral_cluster_model import SHUFFLE_METHODS
SHUFFLE_METHODS = ['layer', 'layer_nonzero']

In [5]:
N_TRAINED_MODELS = 10

# we ar doing 320 experiments (=32 models x 10 times),
# and let's have expectation of false-positive <= 1
# so we can set n_shuffles to 320, and then p-values < 1/401
# if we make it too large, let see, 1000 shuffles...
# ... we might reduce the statistical power (given H1, the probability to reject it incorrectly)!
N_SHUFFLES = 320

N_GPUS = 4

TAGS_COMMANDS = {
    'MNIST': 'python -m src.train_nn with mlp_config dataset_name=mnist',
    'CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=cifar10 pruning_epochs=40',
#    'LINE': 'python -m src.train_nn with mlp_config dataset_name=line',
    'FASHION': 'python -m src.train_nn with mlp_config dataset_name=fashion',
    'MNIST+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist with_dropout=True',
    'CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=cifar10 epochs=100 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'LINE+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line with_dropout=True',
    'FASHION+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=fashion with_dropout=True',
#    'LINE-MNIST': 'python -m src.train_nn with mlp_config dataset_name=line-mnist',
#    'LINE-CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10 epochs=30 pruning_epochs=40',
#    'MNIST-CIFAR10': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10 epochs=30 pruning_epochs=40',
#    'LINE-MNIST-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=line-mnist-separated',
#    'LINE-CIFAR10-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10-separated epochs=30 pruning_epochs=40',
#    'MNIST-CIFAR10-SEPARATED': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10-separated epochs=30 pruning_epochs=40',
#    'LINE-MNIST+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-mnist with_dropout=True',
#    'LINE-CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10 epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'MNIST-CIFAR10+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10 epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'LINE-MNIST-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-mnist-separated with_dropout=True',
#    'LINE-CIFAR10-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=line-cifar10-separated epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'MNIST-CIFAR10-SEPARATED+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist-cifar10-separated epochs=30 pruning_epochs=40 with_dropout=True dropout_rate=0.2',
#    'RANDOM': 'python -m src.train_nn with mlp_config dataset_name=random',
#    'RANDOM+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random with_dropout=True',
#    'MNIST-x1.5-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=30',
#    'MNIST-x1.5-EPOCHS+DROPOUT':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=30 with_dropout=True',
#    'MNIST-x2-EPOCHS':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=40',
#    'MNIST-x2-EPOCHS+DROPOUT':'python -m src.train_nn with mlp_config dataset_name=mnist epochs=40 with_dropout=True',
#    'MNIST-x10-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=200',
#    'MNIST-x10-EPOCHS+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=mnist epochs=200 with_dropout=True',
#    'RANDOM-x50-EPOCHS': 'python -m src.train_nn with mlp_config dataset_name=random epochs=1000',
#    'RANDOM-x50-EPOCHS+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random epochs=1000 with_dropout=True',
#    'RANDOM-OVERFITTING': 'python -m src.train_nn with mlp_config dataset_name=random epochs=100 pruning_epochs=100 shuffle=False n_train=3000',
#    'RANDOM-OVERFITTING+DROPOUT': 'python -m src.train_nn with mlp_config dataset_name=random epochs=100 pruning_epochs=100 shuffle=False n_train=3000 with_dropout=True'
}
DATASETS_TAGS = [command.split()[5][13:] for command in TAGS_COMMANDS.values()]

In [6]:
if False:
    def train_model_multiply(command, n_models_per_command, gpu_id=None):    
        for _ in range(n_models_per_command):
            actual_command = f'cd .. && {command}'#' > /dev/null 2>&1'
            print(actual_command)
            subprocess.run(actual_command, shell=True,
                           env={'CUDA_VISIBLE_DEVICES': str(gpu_id)})


    def train_models(commands, n_models_per_command, gpu_id=None):
        for command in commands:
            train_model_multiply(command, n_models_per_command, gpu_id)

            commands = list(TAGS_COMMANDS.values())
    random.shuffle(commands)

    assert len(commands) % N_GPUS == 0
    n_commands_per_gpu = len(commands) // N_GPUS

    with Pool(N_GPUS) as p:

        p.starmap(train_models,
                 [(commands[gpu_id*n_commands_per_gpu : (gpu_id+1)*n_commands_per_gpu], N_TRAINED_MODELS, gpu_id)
                 for gpu_id in range(N_GPUS)])

In [7]:
results = {}

for shuffle_method in SHUFFLE_METHODS:
    results[shuffle_method] = {}
    
    for (model_tag, _), dataset_tag in zip(TAGS_COMMANDS.items(), tqdm(DATASETS_TAGS)):
        model_paths = get_model_path(model_tag, filter_='all', model_base_path='/scratch/shlomi/models/')


        results[shuffle_method][model_tag] = [run_spectral_cluster(path / f'{dataset_tag}-mlp-pruned-weights.pckl',
                                               n_samples=N_SHUFFLES,
                                               shuffle_method=shuffle_method,
                                               n_clusters=N_CLUSTERS)
            for path in tqdm(model_paths[-N_TRAINED_MODELS:])]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [04:09<37:29, 249.97s/it]

 20%|██        | 2/10 [08:14<33:07, 248.45s/it]

 30%|███       | 3/10 [12:17<28:46, 246.61s/it]

...

 70%|███████   | 7/10 [22:35<09:40, 193.54s/it]

 80%|████████  | 8/10 [25:47<06:26, 193.11s/it]

 90%|█████████ | 9/10 [29:02<03:13, 193.71s/it]

100%|██████████| 10/10 [32:17<00:00, 193.85s/it]

100%|██████████| 10/10 [32:17<00:00, 193.70s/it]

In [8]:
run_spectral_cluster?

In [9]:
def build_all_models_per_shuffle_method_df(single_shuffle_method_results):
    model_dfs = []

    for model_tag, model_results in single_shuffle_method_results.items():
        _, metrics  = zip(*model_results)
        model_dfs.append(pd.DataFrame(metrics).assign(model=model_tag))

    df = pd.concat(model_dfs, ignore_index=True)

    df['is_sig'] = np.isclose(df['percentile'], 1 / (N_SHUFFLES + 1))
    
    return df

layer_df = build_all_models_per_shuffle_method_df(results['layer'])
layer_nonzero_df = build_all_models_per_shuffle_method_df(results['layer_nonzero'])

In [10]:
layer_df.head()

,ave_in_out,mean,n_samples,ncut,percentile,stdev,test_acc,test_loss,train_acc,train_loss,z_score,model,is_sig
0,0.199330,7.371774,320,7.149699,0.003115,0.035005,0.9835,0.095332,1.0,0.000048,-6.344132,MNIST,True
1,0.198384,7.390065,320,7.159383,0.003115,0.031648,0.9847,0.090697,1.0,0.000056,-7.288987,MNIST,True
2,0.207732,7.391867,320,7.064823,0.003115,0.036177,0.9830,0.099833,1.0,0.000059,-9.040050,MNIST,True
3,0.199816,7.380998,320,7.144735,0.003115,0.031190,0.9828,0.099546,1.0,0.000062,-7.574911,MNIST,True
4,0.198309,7.409862,320,7.160156,0.003115,0.030076,0.9845,0.097756,1.0,0.000081,-8.302590,MNIST,True


In [11]:
layer_nonzero_df.head()

,ave_in_out,mean,n_samples,ncut,percentile,stdev,test_acc,test_loss,train_acc,train_loss,z_score,model,is_sig
0,0.199330,7.261299,320,7.149699,0.003115,0.043346,0.9835,0.095332,1.0,0.000048,-2.574603,MNIST,True
1,0.198384,7.320522,320,7.159383,0.003115,0.089845,0.9847,0.090697,1.0,0.000056,-1.793533,MNIST,True
2,0.207732,7.161313,320,7.064823,0.003115,0.018134,0.9830,0.099833,1.0,0.000059,-5.320789,MNIST,True
3,0.199816,7.249981,320,7.144735,0.003115,0.059685,0.9828,0.099546,1.0,0.000062,-1.763372,MNIST,True
4,0.198309,7.301281,320,7.160156,0.003115,0.061875,0.9845,0.097756,1.0,0.000081,-2.280814,MNIST,True


In [12]:
MODEL_DATA_COLUMNS = ['model',
                      'train_acc', 'train_loss', 'test_acc', 'test_loss',
                      'ncut', 'ave_in_out',
                      'n_samples']

assert layer_df[MODEL_DATA_COLUMNS].equals(layer_nonzero_df[MODEL_DATA_COLUMNS])

SHUFFLING_DATA_COLUMNS = ['mean', 'percentile', 'stdev', 'z_score', 'is_sig']

two_shuffling_method_df = pd.merge(layer_df[SHUFFLING_DATA_COLUMNS],
                                     layer_nonzero_df[SHUFFLING_DATA_COLUMNS],
                                     left_index=True, right_index=True,
                                     suffixes=('_layer', '_layer_nonzero'))

assert layer_df[MODEL_DATA_COLUMNS].equals(layer_nonzero_df[MODEL_DATA_COLUMNS])

df = pd.merge(layer_df[MODEL_DATA_COLUMNS], two_shuffling_method_df,
              left_index=True, right_index=True,)

df['cohen_d'] = df.apply(lambda r: cohen_d_stats(r['mean_layer'], r['stdev_layer'], r['n_samples'],
                                 r['mean_layer_nonzero'], r['stdev_layer_nonzero'], r['n_samples']),
                                axis=1)

df

,model,train_acc,train_loss,test_acc,test_loss,ncut,ave_in_out,n_samples,mean_layer,percentile_layer,stdev_layer,z_score_layer,is_sig_layer,mean_layer_nonzero,percentile_layer_nonzero,stdev_layer_nonzero,z_score_layer_nonzero,is_sig_layer_nonzero,cohen_d
0,MNIST,1.000000,0.000048,0.9835,0.095332,7.149699,0.199330,320,7.371774,0.003115,0.035005,-6.344132,True,7.261299,0.003115,0.043346,-2.574603,True,0.896282
1,MNIST,1.000000,0.000056,0.9847,0.090697,7.159383,0.198384,320,7.390065,0.003115,0.031648,-7.288987,True,7.320522,0.003115,0.089845,-1.793533,True,0.808496
2,MNIST,1.000000,0.000059,0.9830,0.099833,7.064823,0.207732,320,7.391867,0.003115,0.036177,-9.040050,True,7.161313,0.003115,0.018134,-5.320789,True,0.970519
3,MNIST,1.000000,0.000062,0.9828,0.099546,7.144735,0.199816,320,7.380998,0.003115,0.031190,-7.574911,True,7.249981,0.003115,0.059685,-1.763372,True,0.865591
4,MNIST,1.000000,0.000081,0.9845,0.097756,7.160156,0.198309,320,7.409862,0.003115,0.030076,-8.302590,True,7.301281,0.003115,0.061875,-2.280814,True,0.860478
5,MNIST,1.000000,0.000048,0.9835,0.095626,7.106376,0.203593,320,7.423381,0.003115,0.030406,-10.425870,True,7.357445,0.003115,0.083587,-3.003673,True,0.819586
6,MNIST,1.000000,0.000056,0.9838,0.087238,7.110218,0.203213,320,7.381650,0.003115,0.026138,-10.384694,True,7.323444,0.065421,0.160001,-1.332656,False,0.716328
7,MNIST,1.000000,0.000061,0.9838,0.100444,7.156789,0.198637,320,7.396530,0.003115,0.031746,-7.551794,True,7.323966,0.003115,0.080510,-2.076481,True,0.824012
8,MNIST,1.000000,0.000047,0.9828,0.093142,7.105069,0.203723,320,7.400648,0.003115,0.040944,-7.219160,True,7.264884,0.003115,0.079261,-2.016325,True,0.830063
9,MNIST,1.000000,0.000049,0.9833,0.102283,7.216584,0.192849,320,7.420351,0.003115,0.032120,-6.343847,True,7.364379,0.003115,0.060175,-2.456112,True,0.859223


In [13]:
# df = pd.read_csv('../results/mlp-clustering-stability-two-shuffling-methods-all-samples.csv' index_col=0)

In [14]:
df_grpby_model = df.groupby('model')

TWO_SHUFFLE_STATS_COLUMNS = [f'{stat}_{shuffle_method}'
                             for stat in ['mean', 'stdev', 'z_score', 'percentile', 'is_sig']
                             for shuffle_method in SHUFFLE_METHODS]

statistics_df = pd.concat([(df_grpby_model[['ncut'] + TWO_SHUFFLE_STATS_COLUMNS]
                             .agg(['mean', 'std'])),
                            df_grpby_model.size().rename('n_models'),
                            (df_grpby_model[['train_acc', 'test_acc']]
                             .agg('mean'))],
                           axis=1)

In [15]:
statistics_df

,"(ncut, mean)","(ncut, std)","(mean_layer, mean)","(mean_layer, std)","(mean_layer_nonzero, mean)","(mean_layer_nonzero, std)","(stdev_layer, mean)","(stdev_layer, std)","(stdev_layer_nonzero, mean)","(stdev_layer_nonzero, std)",...,"(percentile_layer, std)","(percentile_layer_nonzero, mean)","(percentile_layer_nonzero, std)","(is_sig_layer, mean)","(is_sig_layer, std)","(is_sig_layer_nonzero, mean)","(is_sig_layer_nonzero, std)",n_models,train_acc,test_acc
model,,,,,,,,,,,,,,,,,,,,,
CIFAR10,7.330916,0.124631,7.302835,0.009845,7.311578,0.103947,0.029222,0.004019,0.120705,0.110087,...,0.424561,0.445483,0.358459,0.2,0.421637,0.1,0.316228,10,0.650088,0.41523
CIFAR10+DROPOUT,6.972755,0.168614,7.298376,0.045351,7.152856,0.161490,0.020789,0.001996,0.152064,0.025095,...,0.000000,0.165109,0.114509,1.0,0.000000,0.2,0.421637,10,0.427050,0.42189
FASHION,6.975436,0.116762,7.256516,0.020824,7.050758,0.066956,0.029462,0.003895,0.128125,0.033374,...,0.009851,0.376947,0.336166,0.9,0.316228,0.1,0.316228,10,0.982630,0.89344
FASHION+DROPOUT,6.459626,0.040647,7.339441,0.012178,6.731609,0.060870,0.032107,0.004971,0.085221,0.034940,...,0.000000,0.003115,0.000000,1.0,0.000000,1.0,0.000000,10,0.863268,0.86921
MNIST,7.137383,0.041987,7.396713,0.017068,7.292851,0.060505,0.032545,0.004013,0.073642,0.037136,...,0.000000,0.009346,0.019703,1.0,0.000000,0.9,0.316228,10,1.000000,0.98357
MNIST+DROPOUT,6.688336,0.034964,7.421061,0.005184,6.881101,0.021789,0.030407,0.003665,0.034651,0.031148,...,0.000000,0.003115,0.000000,1.0,0.000000,1.0,0.000000,10,0.966788,0.97940


In [16]:
# TODO: ask for a code review
# std = np.sqrt(np.mean(x**2) - np.mean(x)**2)

def calc_overall_stdev(row, suffix=''):
    return np.sqrt(                 # 5. apply root square to get the SD from the variance
        (np.mean(                   # 3. divide by the number of trained model
            (row[f'stdev{suffix}']**2)       # 1. cancel the overall square root
             + row[f'mean{suffix}']**2))     # 2. add the mean and get the sum or squares
        - np.mean(row[f'mean{suffix}'])**2)  # 4. substrct the square of the overall mean

In [17]:
SHUFFLE_METHOD_SUFFIX = ['_layer', '_layer_nonzero']
for suffix in SHUFFLE_METHOD_SUFFIX:
    statistics_df[f'overall_std{suffix}'] = df_grpby_model.apply(calc_overall_stdev, suffix)

for suffix in SHUFFLE_METHOD_SUFFIX:
    statistics_df[f'is_stable{suffix}'] = (statistics_df[(f'is_sig{suffix}', 'mean')] == 1)

statistics_df = statistics_df.loc[TAGS_COMMANDS.keys(), :]

In [18]:
statistics_df

,"(ncut, mean)","(ncut, std)","(mean_layer, mean)","(mean_layer, std)","(mean_layer_nonzero, mean)","(mean_layer_nonzero, std)","(stdev_layer, mean)","(stdev_layer, std)","(stdev_layer_nonzero, mean)","(stdev_layer_nonzero, std)",...,"(is_sig_layer, std)","(is_sig_layer_nonzero, mean)","(is_sig_layer_nonzero, std)",n_models,train_acc,test_acc,overall_std_layer,overall_std_layer_nonzero,is_stable_layer,is_stable_layer_nonzero
model,,,,,,,,,,,,,,,,,,,,,
MNIST,7.137383,0.041987,7.396713,0.017068,7.292851,0.060505,0.032545,0.004013,0.073642,0.037136,...,0.000000,0.9,0.316228,10,1.000000,0.98357,0.036549,0.099795,True,False
CIFAR10,7.330916,0.124631,7.302835,0.009845,7.311578,0.103947,0.029222,0.004019,0.120705,0.110087,...,0.421637,0.1,0.316228,10,0.650088,0.41523,0.030915,0.187620,False,False
FASHION,6.975436,0.116762,7.256516,0.020824,7.050758,0.066956,0.029462,0.003895,0.128125,0.033374,...,0.316228,0.1,0.316228,10,0.982630,0.89344,0.035664,0.146470,False,False
MNIST+DROPOUT,6.688336,0.034964,7.421061,0.005184,6.881101,0.021789,0.030407,0.003665,0.034651,0.031148,...,0.000000,1.0,0.000000,10,0.966788,0.97940,0.030998,0.050012,True,True
CIFAR10+DROPOUT,6.972755,0.168614,7.298376,0.045351,7.152856,0.161490,0.020789,0.001996,0.152064,0.025095,...,0.000000,0.2,0.421637,10,0.427050,0.42189,0.047820,0.217166,True,False
FASHION+DROPOUT,6.459626,0.040647,7.339441,0.012178,6.731609,0.060870,0.032107,0.004971,0.085221,0.034940,...,0.000000,1.0,0.000000,10,0.863268,0.86921,0.034446,0.108148,True,True


In [19]:
df.to_csv(f'../results/mlp-clustering-stability-two-shuffling-methods-all-samples-K-{N_CLUSTERS}.csv')
statistics_df.to_csv(f'../results/mlp-clustering-stability-two-shuffling-methods-statistics-K-{N_CLUSTERS}.csv')

### Model Stability Statistics

In [20]:
statistics_df

,"(ncut, mean)","(ncut, std)","(mean_layer, mean)","(mean_layer, std)","(mean_layer_nonzero, mean)","(mean_layer_nonzero, std)","(stdev_layer, mean)","(stdev_layer, std)","(stdev_layer_nonzero, mean)","(stdev_layer_nonzero, std)",...,"(is_sig_layer, std)","(is_sig_layer_nonzero, mean)","(is_sig_layer_nonzero, std)",n_models,train_acc,test_acc,overall_std_layer,overall_std_layer_nonzero,is_stable_layer,is_stable_layer_nonzero
model,,,,,,,,,,,,,,,,,,,,,
MNIST,7.137383,0.041987,7.396713,0.017068,7.292851,0.060505,0.032545,0.004013,0.073642,0.037136,...,0.000000,0.9,0.316228,10,1.000000,0.98357,0.036549,0.099795,True,False
CIFAR10,7.330916,0.124631,7.302835,0.009845,7.311578,0.103947,0.029222,0.004019,0.120705,0.110087,...,0.421637,0.1,0.316228,10,0.650088,0.41523,0.030915,0.187620,False,False
FASHION,6.975436,0.116762,7.256516,0.020824,7.050758,0.066956,0.029462,0.003895,0.128125,0.033374,...,0.316228,0.1,0.316228,10,0.982630,0.89344,0.035664,0.146470,False,False
MNIST+DROPOUT,6.688336,0.034964,7.421061,0.005184,6.881101,0.021789,0.030407,0.003665,0.034651,0.031148,...,0.000000,1.0,0.000000,10,0.966788,0.97940,0.030998,0.050012,True,True
CIFAR10+DROPOUT,6.972755,0.168614,7.298376,0.045351,7.152856,0.161490,0.020789,0.001996,0.152064,0.025095,...,0.000000,0.2,0.421637,10,0.427050,0.42189,0.047820,0.217166,True,False
FASHION+DROPOUT,6.459626,0.040647,7.339441,0.012178,6.731609,0.060870,0.032107,0.004971,0.085221,0.034940,...,0.000000,1.0,0.000000,10,0.863268,0.86921,0.034446,0.108148,True,True


### Appendix - All Samples

In [21]:
df

,model,train_acc,train_loss,test_acc,test_loss,ncut,ave_in_out,n_samples,mean_layer,percentile_layer,stdev_layer,z_score_layer,is_sig_layer,mean_layer_nonzero,percentile_layer_nonzero,stdev_layer_nonzero,z_score_layer_nonzero,is_sig_layer_nonzero,cohen_d
0,MNIST,1.000000,0.000048,0.9835,0.095332,7.149699,0.199330,320,7.371774,0.003115,0.035005,-6.344132,True,7.261299,0.003115,0.043346,-2.574603,True,0.896282
1,MNIST,1.000000,0.000056,0.9847,0.090697,7.159383,0.198384,320,7.390065,0.003115,0.031648,-7.288987,True,7.320522,0.003115,0.089845,-1.793533,True,0.808496
2,MNIST,1.000000,0.000059,0.9830,0.099833,7.064823,0.207732,320,7.391867,0.003115,0.036177,-9.040050,True,7.161313,0.003115,0.018134,-5.320789,True,0.970519
3,MNIST,1.000000,0.000062,0.9828,0.099546,7.144735,0.199816,320,7.380998,0.003115,0.031190,-7.574911,True,7.249981,0.003115,0.059685,-1.763372,True,0.865591
4,MNIST,1.000000,0.000081,0.9845,0.097756,7.160156,0.198309,320,7.409862,0.003115,0.030076,-8.302590,True,7.301281,0.003115,0.061875,-2.280814,True,0.860478
5,MNIST,1.000000,0.000048,0.9835,0.095626,7.106376,0.203593,320,7.423381,0.003115,0.030406,-10.425870,True,7.357445,0.003115,0.083587,-3.003673,True,0.819586
6,MNIST,1.000000,0.000056,0.9838,0.087238,7.110218,0.203213,320,7.381650,0.003115,0.026138,-10.384694,True,7.323444,0.065421,0.160001,-1.332656,False,0.716328
7,MNIST,1.000000,0.000061,0.9838,0.100444,7.156789,0.198637,320,7.396530,0.003115,0.031746,-7.551794,True,7.323966,0.003115,0.080510,-2.076481,True,0.824012
8,MNIST,1.000000,0.000047,0.9828,0.093142,7.105069,0.203723,320,7.400648,0.003115,0.040944,-7.219160,True,7.264884,0.003115,0.079261,-2.016325,True,0.830063
9,MNIST,1.000000,0.000049,0.9833,0.102283,7.216584,0.192849,320,7.420351,0.003115,0.032120,-6.343847,True,7.364379,0.003115,0.060175,-2.456112,True,0.859223


### Unpruned results without shuffling
#### TODO: Refactor
- [ ] Combine with the unpruned table

In [22]:
if False:
    unpruned_results = {}

    for (model_tag, _), dataset_tag in zip(TAGS_COMMANDS.items(), tqdm(DATASETS_TAGS)):
        model_paths = get_model_path(model_tag, filter_='all', model_base_path='/scratch/shlomi/models/')
        unpruned_results[model_tag] = [run_spectral_cluster(path / f'{dataset_tag}-mlp-unpruned-weights.pckl',
                                                   with_shuffle=False)
                for path in tqdm(model_paths[-N_TRAINED_MODELS:])]

    unpruned_model_dfs = []

    for model_tag, model_results in unpruned_results.items():
        _, metrics  = zip(*model_results)
        unpruned_model_dfs.append(pd.DataFrame(metrics).assign(model=model_tag))

    unpruned_df = pd.concat(unpruned_model_dfs, ignore_index=True)

    unpruned_df_grpby_model = unpruned_df.groupby('model')


    unpruned_statistics_df = pd.concat([(unpruned_df_grpby_model[['ncut']]
                                 .agg(['mean', 'std'])),
                                unpruned_df_grpby_model.size().rename('n_models'),
                                (unpruned_df_grpby_model[['train_acc', 'test_acc']]
                                 .agg('mean'))],
                               axis=1)

    unpruned_statistics_df = unpruned_statistics_df.loc[TAGS_COMMANDS.keys(), :]

    unpruned_statistics_df

In [23]:
# unpruned_df.to_csv(f'../results/mlp-clustering-stability-all-samples-unpruned-K-{N_CLUSTERS}.csv')
# unpruned_statistics_df.to_csv(f'../results/mlp-clustering-stability-statistics-unpruned-K-{N_CLUSTERS}.csv')